In [1]:
# Clone the LeRobot repository from GitHub using git
!git clone https://github.com/huggingface/lerobot.git

Cloning into 'lerobot'...
remote: Enumerating objects: 44845, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 44845 (delta 43), reused 40 (delta 38), pack-reused 44784 (from 3)
Receiving objects: 100% (44845/44845), 220.92 MiB | 19.78 MiB/s, done.
Resolving deltas: 100% (28956/28956), done.
Filtering content: 100% (45/45), 69.03 MiB | 23.20 MiB/s, done.


In [2]:
# Change the current working directory to the 'lerobot' folder
%cd lerobot

/Users/ryosuketakanami/workspace/ihr/rfm_tutorial/lerobot


In [6]:
# Install the current package along with the optional 'pusht' dependencies
!pip install ".[pusht]"

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Processing /Users/ryosuketakanami/workspace/ihr/rfm_tutorial/lerobot
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached datasets-4.1.1-py3-none-any.whl.metadata (18 kB)
  Using cached diffusers-0.35.2-py3-none-any.whl.metadata (20 kB)
  Using cached huggingface_hub-0.35.3-py3-none-any.whl.metadata (14 kB)
  Using cached accelerate-1.10.1-py3-none-any.whl.metadata (19 kB)
  Using cached cmake-4.1.3-py3-none-macosx_10_10_universal2.whl.metadata (6.5 kB)
  Using cached einops-0.8.1-py3-none-any.whl.metadata (13 kB)
  Using cached opencv_python_headless-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl.metadata (19 kB)
  Using cached av-15.1.0-c

In [ ]:
# Install the 'hf_xet' package
!pip install hf_xet

In [12]:
# Run the pretrained Diffusion Policy model on the Push-T task
!python examples/tutorial/diffusion/diffusion_training_example.py

2025-12-11 16:00:12.942295: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765468813.186702    2568 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765468813.254410    2568 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765468813.746367    2568 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765468813.746410    2568 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765468813.746414    2568 computation_placer.cc:177] computation placer alr

In [14]:
import os

script_path = 'examples/tutorial/diffusion/diffusion_training_example.py'

# Read the original script content
with open(script_path, 'r') as f:
    content = f.read()

# Replace 'mps' related device assignments with 'cuda'
# This handles common patterns like device = torch.device("mps") or device = "mps"
content = content.replace('torch.device("mps")', 'torch.device("cuda")')
content = content.replace('device = "mps"', 'device = "cuda"')

# If a conditional check for MPS exists, replace it to use CUDA instead
# This makes sure that device availability checks also switch from mps to cuda
content = content.replace('torch.backends.mps.is_available()', 'torch.cuda.is_available()')

# Write the modified content back to the file
with open(script_path, 'w') as f:
    f.write(content)

print(f"Modified {script_path} to use 'cuda' device.")

Modified examples/tutorial/diffusion/diffusion_training_example.py to use 'cuda' device.


Now that the script has been modified to use 'cuda', let's try running the diffusion training example again.

In [15]:
# Run the pretrained Diffusion Policy model on the Push-T task
!python examples/tutorial/diffusion/diffusion_training_example.py

2025-12-11 16:03:39.824013: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765469019.845372    3526 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765469019.852468    3526 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765469019.870463    3526 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765469019.870494    3526 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765469019.870499    3526 computation_placer.cc:177] computation placer alr

In [13]:
# Train the Diffusion Policy on the Push-T task
!python examples/tutorial/diffusion/diffusion_using_example.py

2025-12-11 16:00:38.386922: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765468838.406540    2744 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765468838.413217    2744 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765468838.428732    2744 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765468838.428758    2744 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765468838.428761    2744 computation_placer.cc:177] computation placer alr